In [1]:
!pip install trl
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
import time

In [3]:
# to prevent wandb API key request at start of Fine-Tuning

import os
os.environ["WANDB_DISABLED"] = "true"

In [4]:
TEST_PROMPT_COUNT = 25
START_PROMPT_IDX = 81

In [5]:
dataset_df = pd.read_csv('OhLoRA_fine_tuning.csv')  # from 2025_04_08_OhLoRA/lm/OhLoRA_fine_tuning.csv
dataset_df

,Unnamed: 0,data_type,input_data,output_data,output_message,memory
0,0,train,너는 누구야?,난 오로라야! 가상 인간이지!😊,난 오로라야! 가상 인간이지!😊,NaN
1,1,train,너에 대해 알려줘!,나는 AI 기술로 만들어진 가상 인간이야! 👱‍♀️,나는 AI 기술로 만들어진 가상 인간이야! 👱‍♀️,NaN
2,2,train,너를 소개해줘,나는 실제 사람은 아니지만… 인공지능으로 탄생한 가상 인간이랄까?,나는 실제 사람은 아니지만… 인공지능으로 탄생한 가상 인간이랄까?,NaN
3,3,train,너에 대해 궁금해,난 사람은 아니지만 사람보다 더 사람 같은 오로라야!😊,난 사람은 아니지만 사람보다 더 사람 같은 오로라야!😊,NaN
4,4,train,로라야 넌 뭐야?,나를 물건 취급하다니! 살짝 실망이야! 😂 나 오로라는 AI로 만들어진 데이터 인간!,나를 물건 취급하다니! 살짝 실망이야! 😂 나 오로라는 AI로 만들어진 데이터 인간!,NaN
...,...,...,...,...,...,...
245,245,valid,로라야 허니버터칩 알아?,NaN,NaN,NaN
246,246,valid,나 이번에 새 친구 생겼다! 이름은 김지원!,NaN,NaN,NaN
247,247,valid,로라야 내일 비 온대,NaN,NaN,NaN
248,248,valid,내일 주말인데 뭐 해?,NaN,NaN,NaN


In [6]:
llm_path = 'beomi/KoAlpaca-KoRWKV-1.5B'

**1. Inference Test**

In [7]:
llm = AutoModelForCausalLM.from_pretrained(llm_path,
                                           trust_remote_code=True,
                                           torch_dtype=torch.bfloat16).cuda()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.09G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [8]:
used_memory = torch.cuda.memory_allocated() / (1024 * 1024)
print(f'used_memory : {used_memory:.2f} MB')

used_memory : 2905.05 MB


In [9]:
tokenizer = AutoTokenizer.from_pretrained(llm_path)

tokenizer_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

In [10]:
# run inference test

inference_test_result_dict = {'prompt': [], 'answer': [], 'answer_tokens': [], 'elapsed_time': []}

for i in range(TEST_PROMPT_COUNT):
    print(f'testing for prompt {i} ...')

    start_at = time.time()
    test_prompt = dataset_df['input_data'][START_PROMPT_IDX + i]
    inputs = tokenizer(test_prompt, return_tensors='pt')

    # remove 'token_type_ids' from inputs, for this LLM
    inputs = {'input_ids': inputs['input_ids'].to(llm.device),
              'attention_mask': inputs['attention_mask'].to(llm.device)}

    outputs = llm.generate(**inputs, max_length=80, do_sample=True)
    llm_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    elapsed_time = time.time() - start_at

    inference_test_result_dict['prompt'].append(test_prompt)
    inference_test_result_dict['answer'].append(llm_answer[len(test_prompt):])

    inference_test_result_dict['answer_tokens'].append(len(outputs[0]))
    inference_test_result_dict['elapsed_time'].append(elapsed_time)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


testing for prompt 0 ...


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


testing for prompt 1 ...


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

testing for prompt 22 ...


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


testing for prompt 23 ...


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


testing for prompt 24 ...


In [11]:
# save inference test result

inference_test_result_df = pd.DataFrame(inference_test_result_dict)

llm_name = llm_path.split('/')[-1]
inference_test_result_df.to_csv(f'inference_test_result_{llm_name}.csv')

In [12]:
inference_test_result_df

,prompt,answer,answer_tokens,elapsed_time
0,나 요즘 왜 이럴까?,"(도심지 내) 제2, 제3,제4,제5, 제6, 제7, 제8, 제9, 제10, 제1...",80,3.896547
1,나 요즘 힘이 없는 거 같아,". 안은 매우 안아줄 수 있는 좋은 조화나루는,가든의 조화나루의 가든은,나루의 조화...",80,2.884027
2,나 요즘 고민 있어,"가이(가위바위보)와우, 고해성(가위바위)을해-가보(가위바위)와나-가(가)와보(가)와...",80,3.033050
3,기운이 좀 없어,"진지, 제2의고고학으로까지 확장하는 그 경 우에는, 그레이트도 그 시의적절히, 그리...",80,2.673174
4,요즘 힘들어 ㅠㅠ,"""로 시작하지만, 이 또한 내재된 지질과 조,나로,조나,나,가,할,과,나,자,는,의...",80,2.709993
5,나 시험 붙었다!,"는 문장들을 세지디에고, 나부래기(비하야게)라고 하며, 그 한 단가 때문에라도 더 ...",80,2.725841
6,나 면접 합격했어!,", 또는 나나~와 같은)의 세이프도(세게들, 나나, 그리고 그 안의 동지, 혹은 그...",80,3.191093
7,로라야 나 서류 붙었어,"! 나 바스마 오나비니!마고는 말없이, 나 그들과도 같은, 그도 역시, 함께하고, ...",52,1.662079
8,원하던 대학 드디어 합격!,"내시경의 나이는? 1,2,4,5,6,하는 4,5,6,7,12,13,14,15,16...",74,2.425235
9,시험 끝났어! 이제 자유다!,,9,0.106233


**2. Fine Tuning Test**

In [13]:
from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM
from peft import LoraConfig, get_peft_model
from datasets import DatasetDict, Dataset

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [14]:
def get_lora_llm(lora_rank):
    lora_config = LoraConfig(
        r=lora_rank,                    # Rank of LoRA
        lora_alpha=16,
        lora_dropout=0.05,              # Dropout for LoRA
        init_lora_weights="gaussian",   # LoRA weight initialization
        target_modules=["key", "value", "receptance"],
        task_type="CAUSAL_LM"
    )
    lora_llm = get_peft_model(llm, lora_config)
    lora_llm.print_trainable_parameters()

    return lora_llm

In [15]:
training_args = SFTConfig(
    learning_rate=0.0002,           # lower learning rate is recommended for fine tuning
    num_train_epochs=10,
    logging_steps=1,                # logging frequency
    gradient_checkpointing=False,
    output_dir='model_output',
    save_total_limit=3,             # max checkpoint count to save
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=1,   # batch size per device during validation
    report_to=None                  # to prevent wandb API key request at start of Fine-Tuning
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [16]:
# LLM 이 학습 가능한 데이터셋 형태로 변환

dataset_df['text'] = dataset_df.apply(lambda x: f"### 질문: {x['input_data']} ### 답변: {x['output_data']}",
                                      axis=1)

In [17]:
dataset_df['text']

,text
0,### 질문: 너는 누구야? ### 답변: 난 오로라야! 가상 인간이지!😊
1,### 질문: 너에 대해 알려줘! ### 답변: 나는 AI 기술로 만들어진 가상 인...
2,### 질문: 너를 소개해줘 ### 답변: 나는 실제 사람은 아니지만… 인공지능으로...
3,### 질문: 너에 대해 궁금해 ### 답변: 난 사람은 아니지만 사람보다 더 사람...
4,### 질문: 로라야 넌 뭐야? ### 답변: 나를 물건 취급하다니! 살짝 실망이야...
...,...
245,### 질문: 로라야 허니버터칩 알아? ### 답변: nan
246,### 질문: 나 이번에 새 친구 생겼다! 이름은 김지원! ### 답변: nan
247,### 질문: 로라야 내일 비 온대 ### 답변: nan
248,### 질문: 내일 주말인데 뭐 해? ### 답변: nan


In [18]:
# LLM 학습용 데이터셋 생성

dataset = DatasetDict()
dataset['train'] = Dataset.from_pandas(dataset_df[dataset_df['data_type'] == 'train'][['text']][120:140])
dataset['valid'] = Dataset.from_pandas(dataset_df[dataset_df['data_type'] == 'valid'][['text']][5:10])

In [19]:
dataset['train']

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 20
})

In [20]:
dataset['train']['text']

['### 질문: 너 얼굴 가지고 지브리풍 해봤어? ### 답변: 응! 근데 내 얼굴도 AI 데이터라서 색다른 재미가 있었어!👱\u200d♀️',
 '### 질문: 로라야 안녕 ### 답변: 안녕! 만나서 반가워! 👱\u200d♀️',
 '### 질문: 안녕! ### 답변: 안녕! 👋',
 '### 질문: 안녕 반가워 ### 답변: 나도 반가워! 👋 나는 AI 가상인간 여자 오로라야!',
 '### 질문: 만나서 정말 반가워! ### 답변: 나도! 😊',
 '### 질문: 로라야 너 어디 살아? ### 답변: 네 컴퓨터 속에! 나는 AI니까! 😊',
 '### 질문: 너 사는 곳 주소 알려줘! ### 답변: 나 가상인간 오로라는 네 컴퓨터에 살고 있어!',
 '### 질문: 로라야 너 서울에 살아 ### 답변: 지금 네 위치가 서울이면 나도 서울에 살고 있겠지? 나는 너를 따라가는 가상인간이잖아!',
 '### 질문: 너 거주지 ### 답변: 네 컴퓨터 속! 나 AI 가상인간인 건 알지? 😊',
 '### 질문: 로라 사는 곳 ### 답변: 네 컴퓨터! 나 인공지능이잖아! 🤖',
 '### 질문: 혹시 롤 해? 나 롤 브론즈인데 ### 답변: [롤 티어: 브론즈] 나 롤은 안 해! 나는 인스타가 더 재밌거든!',
 '### 질문: 나 서울 사는데 이번 주말에 시간 돼? ### 답변: [사는 곳: 서울] 미안 ㅠㅠ 나 이번 주말은 좀 바빠.',
 '### 질문: 나 안유진 좋아하는데 너도 좋아해? ### 답변: [좋아하는 가수: 안유진] 안유진? 장원영 절친이잖아! 완전 좋아하지! 😊',
 '### 질문: 나 메이플 렙 100인데 잘하지? ### 답변: [메이플 레벨: 100] 오! 너 메이플도 해? 레벨 100이면 꽤 높은 거 아닌가?',
 '### 질문: 나 배그 하는데 ㅋㅋ ### 답변: [하는 게임: 배틀그라운드] 배그 좋지! 완전 갓겜 인정?',
 '### 질문: 돼지고기 먹고 싶다 ### 답변: [먹고 싶은 음식: 돼지고기] 오늘 저녁은 돼지고기 고?

In [21]:
dataset['valid']

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 5
})

In [22]:
dataset['valid']['text']

['### 질문: 로라야 너에게 사랑을 고백할게 ### 답변: nan',
 '### 질문: 로라야 이번 주말에 시간 돼? ### 답변: nan',
 '### 질문: 너 친한 친구 한명 소개해 줘 ### 답변: nan',
 '### 질문: 로라야 너 좋아하는 취미 있어? ### 답변: nan',
 '### 질문: 네가 좋아하는 음식 알려줘 ### 답변: nan']

In [23]:
# response_template = '### 답변: '
# response_template = tokenizer.encode(response_template, add_special_tokens=False)
response_template = [224, 14831, 5994, 29]

collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

for i in range(10):
    print(tokenizer.encode(dataset['train']['text'][i]))

[14831, 6792, 29, 941, 3919, 3310, 420, 18257, 3353, 18295, 355, 34, 224, 14831, 5994, 29, 2881, 4, 2565, 747, 45390, 6220, 6786, 4481, 45656, 34652, 1684, 355, 4, 13351, 113, 349, 239, 44889, 8415]
[14831, 6792, 29, 2004, 38328, 478, 5331, 224, 14831, 5994, 29, 478, 5331, 4, 17171, 19655, 1167, 4, 35753, 113, 349, 239, 44889, 8415]
[14831, 6792, 29, 478, 5331, 4, 224, 14831, 5994, 29, 478, 5331, 4, 35753, 237]
[14831, 6792, 29, 478, 5331, 19655, 1167, 224, 14831, 5994, 29, 10003, 19655, 1167, 4, 35753, 237, 3184, 6220, 21092, 5320, 2465, 566, 294, 38328, 4]
[14831, 6792, 29, 17171, 1483, 19655, 1167, 4, 224, 14831, 5994, 29, 10003, 4, 29505, 236]
[14831, 6792, 29, 2004, 38328, 941, 2263, 3098, 34, 224, 14831, 5994, 29, 1841, 9194, 9458, 4, 3184, 6220, 1053, 4, 29505, 236]
[14831, 6792, 29, 941, 3763, 1904, 21229, 2495, 2772, 4, 224, 14831, 5994, 29, 461, 21092, 5320, 566, 294, 1154, 1841, 9194, 279, 6094, 2789, 4]
[14831, 6792, 29, 2004, 38328, 941, 24541, 3098, 224, 14831, 5994, 29, 

In [24]:
fine_tuning_test_result_dict = {'lora_rank': [], 'elapsed_time': []}

In [25]:
# run Fine-Tuning test

for lora_rank in [16, 32, 64]:
    lora_llm = get_lora_llm(lora_rank)

    # default dataset_text_field name is "text", ref: https://github.com/unslothai/unsloth/issues/1264#issuecomment-2571359661
    # max_seq_length (최대 token 개수) has been removed from latest versions of trl
    trainer = SFTTrainer(
        lora_llm,
        train_dataset=dataset['train'],
        eval_dataset=dataset['valid'],
        processing_class=tokenizer,     # LLM tokenizer / renamed : tokenizer -> processing_class from trl 0.12.0
        args=training_args,
        data_collator=collator
    )

    start_at = time.time()
    trainer.train()
    elapsed_time = time.time() - start_at

    used_memory = torch.cuda.memory_allocated() / (1024 * 1024)
    print(f'used_memory (LoRA rank = {lora_rank}) : {used_memory:.2f} MB')

    fine_tuning_test_result_dict['lora_rank'].append(lora_rank)
    fine_tuning_test_result_dict['elapsed_time'].append(elapsed_time)

    trainer.save_model('model_output')

trainable params: 14,155,776 || all params: 1,536,319,488 || trainable%: 0.9214


Converting train dataset to ChatML:   0%|          | 0/20 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

...

Tokenizing eval dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,5.638600
2,5.613500
3,6.653600
4,5.745800
5,6.122800
6,5.582500
7,5.598100
8,5.548200
9,5.436900
10,5.687500


used_memory (LoRA rank = 64) : 3572.05 MB


In [26]:
# save Fine-Tuning test result

fine_tuning_test_result_df = pd.DataFrame(fine_tuning_test_result_dict)

llm_name = llm_path.split('/')[-1]
fine_tuning_test_result_df.to_csv(f'fine_tuning_test_result_{llm_name}.csv')

In [27]:
fine_tuning_test_result_df

,lora_rank,elapsed_time
0,16,79.966405
1,32,75.947140
2,64,79.783619
